In [ ]:
from __future__ import annotations

import importlib
import sys
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd

sys.path.append(str(Path("__file__").resolve().parent.parent.parent))

from sub import estimate

importlib.reload(estimate)

plt.rcParams["figure.figsize"] = (16.0, 6.0)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

LOG_FILE_PATH = "../../../dataset/sample_trials/4_1_51.txt"
GIS_BASE_PATH = "../../../dataset/gis/"
BEACON_LIST_PATH = GIS_BASE_PATH + "beacon_list.csv"
FLOOR_NAMES = ["FLU01", "FLU02", "FLD01"]
fig_size = (5, 5)
# 1 pixel of bmp represents 0.01 m
dx = 0.01
dy = 0.01

In [ ]:
from typing import Literal

from sub import pdr

Axis2D = Literal["x", "y"]
importlib.reload(pdr)

data = pdr.read_log_data(LOG_FILE_PATH)
acc_df, gyro_df, mgf_df, ground_truth_df, ble_scans_df = pdr.convert_to_dataframes(data)
ble_positions_df = pd.read_csv(BEACON_LIST_PATH)
floor_name = ground_truth_df["floor_name"][0]
map_dict = pdr.load_floor_maps(FLOOR_NAMES, GIS_BASE_PATH)

true_point: dict[Axis2D, float] = {
    "x": ground_truth_df["x"][0],
    "y": ground_truth_df["y"][0],
}

In [ ]:
from sub import pdr, utils

importlib.reload(utils)

angle_df, cumulative_displacement_df = pdr.estimate_trajectory(
    acc_df,
    gyro_df,
    ground_truth_first_point=true_point,
)

utils.plot_displacement_map(
    map_dict,
    floor_name,
    dx,
    dy,
    cumulative_displacement_df,
    fig_size=fig_size,
)

In [ ]:
specific_second = 80
filter_second_cumulative_displacement_df = cumulative_displacement_df[
    cumulative_displacement_df.ts < specific_second
]

utils.plot_displacement_map(
    map_dict,
    floor_name,
    dx,
    dy,
    filter_second_cumulative_displacement_df,
    fig_size=fig_size,
)

In [ ]:
import pandas as pd
from sub import ble, ble_fingerprint

importlib.reload(ble_fingerprint)
importlib.reload(ble)
# 軌跡前半の歩行軌跡の座標と強いBLEビーコンの位置座標の距離が最小になる角度を探索
# これは軌跡前半はドリフトが乗りづらいため
# 時間全体の中央を変数に入れる
center_of_time = ble_scans_df.tail(1)["ts"].to_numpy() / 2

first_half_strong_blescans = estimate.filter_strong_blescans(
    ble_scans_df,
    180,
    -76,
)

In [ ]:
merged_strong_blescans_first_half = estimate.load_beacons_and_merge(
    BEACON_LIST_PATH,
    first_half_strong_blescans,
)

# マップ上にビーコンの位置をプロット
utils.plot_map(map_dict, floor_name, dx, dy, fig_size=fig_size)
plt.colorbar(
    plt.scatter(
        merged_strong_blescans_first_half.x,
        merged_strong_blescans_first_half.y,
        c=merged_strong_blescans_first_half["ts"],
        cmap="rainbow",
        s=5,
    ),
)
plt.xlabel("x(m)")
plt.ylabel("y(m)")
plt.title("Beacons")
plt.show()

In [ ]:
rotated_cumulative_displacement_df = estimate.rotate_cumulative_displacement(
    cumulative_displacement_df,
    0.3,
    {"x": ground_truth_df.x[0], "y": ground_truth_df.y[0]},
)

# 同じ bdaddress, x, y を持つデータをグループ化
merged_df = pd.merge_asof(
    merged_strong_blescans_first_half,
    rotated_cumulative_displacement_df,
    on="ts",
    direction="nearest",
)

# # マップ上にx_displacement,y_displacementとビーコンの位置をプロット
utils.plot_map(map_dict, floor_name, dx, dy, fig_size=fig_size)
plt.colorbar(
    plt.scatter(
        merged_df.x_displacement,
        merged_df.y_displacement,
        c=merged_df["ts"],
        cmap="rainbow",
        s=5,
    ),
)
plt.scatter(merged_df.x, merged_df.y, c="blue", s=5)
plt.show()

In [ ]:
from sub import drift

first_time_remove_drift_angle_df, first_time_remove_drift_angle_displacement = (
    drift.remove_drift_in_angle_df(
        acc_df,
        angle_df,
        ground_truth_df,
    )
)

In [ ]:
import pandas as pd
from sub import ble, ble_fingerprint

importlib.reload(ble_fingerprint)
importlib.reload(ble)

ble_fingerprint_df = pd.read_csv(
    "../../../dataset/ble_finger/beacon_reception_events.csv",
)

(
    rotated_optimal_angle_df,
    rotated_optimal_displacement_df,
    delete_nan_merged_beacon_stats_df,
) = ble_fingerprint.rotate_trajectory_to_optimal_alignment_using_ble_fingerprint(
    acc_df,
    first_time_remove_drift_angle_df,
    ble_scans_df,
    ble_fingerprint_df,
    floor_name,
    ground_truth_first_point={
        "x": ground_truth_df.x[0],
        "y": ground_truth_df.y[0],
    },
)

In [ ]:
importlib.reload(estimate)


rotated_cumulative_displacement_df = estimate.rotate_cumulative_displacement(
    cumulative_displacement_df,
    1.19,
    {"x": ground_truth_df.x[0], "y": ground_truth_df.y[0]},
)


# 同じ bdaddress, x, y を持つデータをグループ化
merged_df = pd.merge_asof(
    delete_nan_merged_beacon_stats_df,
    rotated_cumulative_displacement_df,
    on="ts",
    direction="nearest",
)


merged_df = merged_df[merged_df["ts"] < specific_second]
delete_nan_merged_beacon_stats_df = delete_nan_merged_beacon_stats_df[
    delete_nan_merged_beacon_stats_df["ts"] < specific_second
]

utils.plot_map(map_dict, floor_name, dx, dy, fig_size=(10, 10))

plt.colorbar(
    plt.scatter(
        delete_nan_merged_beacon_stats_df.x,
        delete_nan_merged_beacon_stats_df.y,
        c=delete_nan_merged_beacon_stats_df["ts"],
        cmap="rainbow",
        s=5,
    ),
)
plt.show()

In [ ]:
importlib.reload(estimate)


beacon_address = "C0:1C:4D:47:CF:0A"

delete_nan_merged_beacon_stats_df = delete_nan_merged_beacon_stats_df[
    delete_nan_merged_beacon_stats_df["beacon_address"] == beacon_address
]


utils.plot_map(map_dict, floor_name, dx, dy)
plt.colorbar(
    plt.scatter(
        delete_nan_merged_beacon_stats_df.x,
        delete_nan_merged_beacon_stats_df.y,
        c=delete_nan_merged_beacon_stats_df["ts"],
        cmap="rainbow",
        s=5,
    ),
)
filter_merged_df = merged_df[merged_df["bdaddress"] == beacon_address]
plt.scatter(filter_merged_df.x, filter_merged_df.y, c="blue", s=10)
plt.show()

In [ ]:
utils.plot_map(map_dict, floor_name, dx, dy)
plt.colorbar(
    plt.scatter(
        rotated_optimal_displacement_df.x_displacement,
        rotated_optimal_displacement_df.y_displacement,
        c=rotated_optimal_displacement_df.ts,
        cmap="rainbow",
        s=5,
    ),
)
plt.show()

In [ ]:
from sub import ble

importlib.reload(ble)

rotated_optimal_angle_df, rotated_optimal_displacement_df = (
    ble.rotate_trajectory_to_optimal_alignment_using_ble(
        acc_df,
        first_time_remove_drift_angle_df,
        ble_scans_df,
        ble_positions_df,
        ground_truth_first_point={
            "x": ground_truth_df.x[0],
            "y": ground_truth_df.y[0],
        },
    )
)


estimate.plot_map(map_dict, floor_name, dx, dy)
plt.colorbar(
    plt.scatter(
        rotated_optimal_displacement_df.x_displacement,
        rotated_optimal_displacement_df.y_displacement,
        c=rotated_optimal_displacement_df.ts,
        cmap="rainbow",
        s=5,
    ),
)
plt.xlabel("x_displacement(m)")
plt.ylabel("y_displacement(m)")
plt.title("Displacement")
plt.show()